In [1]:
# Importing the libraries needed
import pandas as pd
import numpy as np
import json
import scipy 
import torch
from torch.utils.data import Dataset, DataLoader
import logging
import os



import preprocessor as p
import transformers
import nltk

# Adding AWS Data Wrangler
import awswrangler as wr

In [2]:
raw_s3_bucket = 'uaresources'
#raw_path_dir = 'aug_Ukraine_War/'

#raw_path = f"s3://{raw_s3_bucket}/time_twitter_data.csv"

df = wr.s3.read_csv(path=['s3://uaresources/time_twitter_data.csv'])


In [3]:
#df = df.toPandas()
df.head()

,following,followers,totaltweets,usercreatedts,tweetcreatedts,retweetcount,text,hashtags,language,favorite_count,is_retweet,is_quote_status,extractedts
0,51,3362,71331,2011-09-08 02:34:54.000000,2022-08-01 00:00:00.000000,0,The #RussiaUkraine conflict is being cast in b...,"[{'text': 'RussiaUkraine', 'indices': [4, 18]}...",en,0.0,False,False,2022-08-01 02:30:13.601013
1,137,114789,315096,2010-08-20 11:19:22.000000,2022-08-01 00:00:01.000000,0,The world is in dire straits as it is not equi...,"[{'text': 'UkraineCrisis', 'indices': [133, 14...",en,1.0,False,False,2022-08-01 01:44:58.884242
2,7284,6963,264990,2009-06-07 05:36:09.000000,2022-08-01 00:00:01.000000,0,Owner of Ukraine's largest grain exporter kill...,"[{'text': 'UkraineInvasion', 'indices': [100, ...",en,1.0,False,False,2022-08-01 02:36:44.414025
3,1080,14429,35346,2013-05-11 18:28:57.000000,2022-08-01 00:00:02.000000,1,Why #Iran-#Russia relations likely won’t go be...,"[{'text': 'Iran', 'indices': [4, 9]}, {'text':...",en,2.0,False,False,2022-08-01 05:13:47.917189
4,1980,1827,67996,2007-03-23 21:01:28.000000,2022-08-01 00:00:02.000000,1,It's happening on live TV #ToryFascistDictator...,"[{'text': 'ToryFascistDictatorship', 'indices'...",en,0.0,True,True,2022-08-01 02:36:44.402793


In [4]:
text_df = df.copy()

In [5]:
# Lowercase everything
text_df["cleaned_text"] = text_df["text"].str.lower()
# check
text_df["cleaned_text"].head(10)

0    the #russiaukraine conflict is being cast in b...
1    the world is in dire straits as it is not equi...
2    owner of ukraine's largest grain exporter kill...
3    why #iran-#russia relations likely won’t go be...
4    it's happening on live tv #toryfascistdictator...
5    serbian vucic is no good,\nalbanistan is no go...
6    russian rocket killed of one of the most dedic...
7    legion freedom of #russia declared that its un...
8    are italian companies massively bypassing #san...
9    turkey refused russia joint production of "bay...
Name: cleaned_text, dtype: object

In [6]:
# Chenge data types
text_df['cleaned_text']=text_df['cleaned_text'].astype(str)

In [7]:
def remove_unnecessary(text):
    # INPUT: string (tweet)
    # OUTPUT: string without URLs, mentions, hashtags, digist, and emojis (and smileys)
    p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.HASHTAG, p.OPT.NUMBER, p.OPT.EMOJI, p.OPT.SMILEY)
    result = p.clean(text)
    return result

text_df["cleaned_text"] = text_df["cleaned_text"].map(remove_unnecessary)
# check
text_df["cleaned_text"].head(10)

0    the conflict is being cast in binaries, making...
1    the world is in dire straits as it is not equi...
2    owner of ukraine's largest grain exporter kill...
3    why - relations likely wont go beyond cooperat...
4                            it's happening on live tv
5    serbian vucic is no good, albanistan is no goo...
6    russian rocket killed of one of the most dedic...
7    legion freedom of declared that its unit now c...
8    are italian companies massively bypassing agai...
9    turkey refused russia joint production of "bay...
Name: cleaned_text, dtype: object

In [8]:
#remove StopWords and see the WordCloud
stopwords_set = set(STOPWORDS)
wordcloud = WordCloud(background_color='white',
                     stopwords = stopwords_set,
                      max_words = 300,
                      max_font_size = 40,
                      scale = 2,
                      random_state=42
                     ).generate(str(text_df['cleaned_text']))

print(wordcloud)
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

NameError: name 'STOPWORDS' is not defined

In [9]:
# replace "&amp" in tweets with empty string
text_df["cleaned_text"] = text_df["cleaned_text"].str.replace("&amp", "")

# replace "\n" in tweets with empty string
# may not be necessary after applying remove_unnecessary func
# feb_df["cleaned_text"] = feb_df["cleaned_text"].str.replace("\n", "")

# check
text_df.head(10)

,following,followers,totaltweets,usercreatedts,tweetcreatedts,retweetcount,text,hashtags,language,favorite_count,is_retweet,is_quote_status,extractedts,cleaned_text
0,51,3362,71331,2011-09-08 02:34:54.000000,2022-08-01 00:00:00.000000,0,The #RussiaUkraine conflict is being cast in b...,"[{'text': 'RussiaUkraine', 'indices': [4, 18]}...",en,0.0,False,False,2022-08-01 02:30:13.601013,"the conflict is being cast in binaries, making..."
1,137,114789,315096,2010-08-20 11:19:22.000000,2022-08-01 00:00:01.000000,0,The world is in dire straits as it is not equi...,"[{'text': 'UkraineCrisis', 'indices': [133, 14...",en,1.0,False,False,2022-08-01 01:44:58.884242,the world is in dire straits as it is not equi...
2,7284,6963,264990,2009-06-07 05:36:09.000000,2022-08-01 00:00:01.000000,0,Owner of Ukraine's largest grain exporter kill...,"[{'text': 'UkraineInvasion', 'indices': [100, ...",en,1.0,False,False,2022-08-01 02:36:44.414025,owner of ukraine's largest grain exporter kill...
3,1080,14429,35346,2013-05-11 18:28:57.000000,2022-08-01 00:00:02.000000,1,Why #Iran-#Russia relations likely won’t go be...,"[{'text': 'Iran', 'indices': [4, 9]}, {'text':...",en,2.0,False,False,2022-08-01 05:13:47.917189,why - relations likely wont go beyond cooperat...
4,1980,1827,67996,2007-03-23 21:01:28.000000,2022-08-01 00:00:02.000000,1,It's happening on live TV #ToryFascistDictator...,"[{'text': 'ToryFascistDictatorship', 'indices'...",en,0.0,True,True,2022-08-01 02:36:44.402793,it's happening on live tv
5,421,87,6591,2013-07-09 15:55:01.000000,2022-08-01 00:00:03.000000,0,"Serbian Vucic is no good,\nAlbanistan is no go...","[{'text': 'kosovo', 'indices': [113, 120]}, {'...",en,0.0,False,False,2022-08-01 00:29:55.970401,"serbian vucic is no good, albanistan is no goo..."
6,478,163,6760,2014-12-26 00:31:46.000000,2022-08-01 00:00:08.000000,76,russian rocket killed of one of the most dedic...,"[{'text': 'Ukraine', 'indices': [81, 89]}, {'t...",en,0.0,True,False,2022-08-01 02:30:13.418591,russian rocket killed of one of the most dedic...
7,155,15,16148,2014-07-29 13:13:34.000000,2022-08-01 00:00:09.000000,282,Legion Freedom of #Russia declared that its un...,"[{'text': 'Russia', 'indices': [18, 25]}]",en,0.0,True,False,2022-08-01 00:51:32.940497,legion freedom of declared that its unit now c...
8,5001,1912,163860,2012-06-23 01:23:23.000000,2022-08-01 00:00:10.000000,454,Are Italian companies massively bypassing #san...,"[{'text': 'sanctions', 'indices': [42, 52]}, {...",en,0.0,True,False,2022-08-01 00:51:32.929302,are italian companies massively bypassing agai...
9,478,59,34296,2009-10-19 22:54:51.000000,2022-08-01 00:00:10.000000,43,"Turkey refused Russia joint production of ""Bay...","[{'text': 'Ukraine', 'indices': [79, 87]}, {'t...",en,0.0,True,False,2022-08-01 01:44:58.780045,"turkey refused russia joint production of ""bay..."


In [10]:
# remove punctuations using regex
# reference: https://stackoverflow.com/questions/68641923/remove-puncts-from-pandas-dataframe
text_df["cleaned_text"] = text_df['cleaned_text'].str.replace(r'[^0-9a-zA-Z\s]+', '', regex=True)

# check
text_df["cleaned_text"].head(15)

0     the conflict is being cast in binaries making ...
1     the world is in dire straits as it is not equi...
2     owner of ukraines largest grain exporter kille...
3     why  relations likely wont go beyond cooperati...
4                              its happening on live tv
5     serbian vucic is no good albanistan is no good...
6     russian rocket killed of one of the most dedic...
7     legion freedom of declared that its unit now c...
8     are italian companies massively bypassing agai...
9     turkey refused russia joint production of bayr...
10    zelensky formal legal recognition of russia as...
11    i condemn brutal murder of ukrainian pows by r...
12     a new video of a combatant during shooting pr...
13                      good night world from with love
14    support for kosovo  all sane people should sho...
Name: cleaned_text, dtype: object

In [11]:
# Drop the old text column as now we have all clean data in the cleaned text
text_df.drop(columns=["text"], inplace=True)

In [12]:
# reset index
text_df.reset_index(inplace=True, drop=True)
# check
text_df.head()

,following,followers,totaltweets,usercreatedts,tweetcreatedts,retweetcount,hashtags,language,favorite_count,is_retweet,is_quote_status,extractedts,cleaned_text
0,51,3362,71331,2011-09-08 02:34:54.000000,2022-08-01 00:00:00.000000,0,"[{'text': 'RussiaUkraine', 'indices': [4, 18]}...",en,0.0,False,False,2022-08-01 02:30:13.601013,the conflict is being cast in binaries making ...
1,137,114789,315096,2010-08-20 11:19:22.000000,2022-08-01 00:00:01.000000,0,"[{'text': 'UkraineCrisis', 'indices': [133, 14...",en,1.0,False,False,2022-08-01 01:44:58.884242,the world is in dire straits as it is not equi...
2,7284,6963,264990,2009-06-07 05:36:09.000000,2022-08-01 00:00:01.000000,0,"[{'text': 'UkraineInvasion', 'indices': [100, ...",en,1.0,False,False,2022-08-01 02:36:44.414025,owner of ukraines largest grain exporter kille...
3,1080,14429,35346,2013-05-11 18:28:57.000000,2022-08-01 00:00:02.000000,1,"[{'text': 'Iran', 'indices': [4, 9]}, {'text':...",en,2.0,False,False,2022-08-01 05:13:47.917189,why relations likely wont go beyond cooperati...
4,1980,1827,67996,2007-03-23 21:01:28.000000,2022-08-01 00:00:02.000000,1,"[{'text': 'ToryFascistDictatorship', 'indices'...",en,0.0,True,True,2022-08-01 02:36:44.402793,its happening on live tv


In [13]:
# get shape of the DataFrame
print(f"{text_df.shape[0]} rows and {text_df.shape[1]} columns")

390649 rows and 13 columns


In [14]:
# Check data types
text_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390649 entries, 0 to 390648
Data columns (total 13 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   following        390649 non-null  int64  
 1   followers        390649 non-null  int64  
 2   totaltweets      390649 non-null  int64  
 3   usercreatedts    390649 non-null  object 
 4   tweetcreatedts   390649 non-null  object 
 5   retweetcount     390649 non-null  int64  
 6   hashtags         390649 non-null  object 
 7   language         390649 non-null  object 
 8   favorite_count   390649 non-null  float64
 9   is_retweet       390649 non-null  bool   
 10  is_quote_status  390649 non-null  bool   
 11  extractedts      390649 non-null  object 
 12  cleaned_text     390649 non-null  object 
dtypes: bool(2), float64(1), int64(4), object(6)
memory usage: 33.5+ MB


Somehow the column tweetcreatedts became an object, we should fix that.

In [15]:
# Convert the time colum in the right type:
text_df["tweetcreatedts"] = pd.to_datetime(text_df["tweetcreatedts"])

# Sentiment Analysis Using RoBERTa

For each tweet the RoBERTa model will generate a score for each of negative, neutral, and positive sentiments

In [16]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Number of GPUs: {torch.cuda.device_count()}")

Number of GPUs: 0


In [17]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

In [18]:
from transformers import AutoModelForSequenceClassification

# Create instance of twitter-roberta-base-sentiment classification model
#model = AutoModel.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

In [19]:
# Attach it to the cuda
model = model.to(device)

In [20]:
import urllib
import csv

labels=[] # will contain 'positive', 'neutral', 'negative'
task = 'sentiment' # our task is sentiment analysis
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

In [21]:
labels

['negative', 'neutral', 'positive']

In [ ]:
from scipy.special import softmax
from tqdm import tqdm

BATCH_SIZE = 100 # number of tweets in a batch that will be passed into tokenizer

scores_all = np.empty((0,len(labels)))
# create list of all the tweets in the dataset
text_df = text_df.dropna()
text_all = text_df['cleaned_text'].tolist()

n = len(text_all) # same as number of tweets
with torch.no_grad():
    for start_idx in tqdm(range(0, n, BATCH_SIZE)):
        end_idx = min(start_idx + BATCH_SIZE, n) 
        # reference: https://huggingface.co/docs/transformers/preprocessing
        # tokenize the tweets in the batch, return pytorch ('pt') tensors
        # some tweets are shorter than the uniform tensor length needed; padding adds 0's to maintain uniform tensor length
        # some tweets are too long; truncation truncates input to maximum length accepted by model
        encoded_input = tokenizer(text_all[start_idx:end_idx], return_tensors='pt', padding=True, truncation=True).to(device)
        # encoded_input = tokenizer(text_all, truncation=True, padding=True)
        # references: https://stackoverflow.com/questions/11315010/what-do-and-before-a-variable-name-mean-in-a-function-signature
        # https://stackoverflow.com/questions/1419046/normal-arguments-vs-keyword-arguments/1419160#1419160
        output = model(**encoded_input)
        # convert pytorch tensor to numpy
        scores = output[0].detach().cpu().numpy()
        # 
        scores = softmax(scores, axis=1)
        scores_all = np.concatenate((scores_all, scores), axis=0)
        
        # delete encoded_input, output, scores for next batch
        del encoded_input, output, scores 
        # release all unoccupied cached mem 
        torch.cuda.empty_cache()

It looks like dataset is too big for our project: estimated time of the analysis (only for the 12 days of August!) is 59 hours. I stopped the sentiment analysis after 359 minutes. However, we have some data.

 Output below is what scores_all looks like for the 12% of finished analysis. Each row contains scores for negative, neutral, and positive sentiments. The higher the score, the more likely the tweet has that sentiment.

In [ ]:
scores_all

In [ ]:
# Let's combine the scores with the existing DataFrame.
text_df[labels] = pd.DataFrame(scores_all, columns=labels)
text_df.head()

In [ ]:
logging.basicConfig(level=logging.ERROR)

In [ ]:
# Save this DataFrame so that we don't have to run the model again, which takes a long time.
file_path = "/Users/olgapodolska/Desktop/UA_War_my/resources/aug_twitter_sentiments.csv"
text_df.to_csv(file_path, index=False)